In [18]:
# External libs
import numpy as np
from pandas import DataFrame as df
import importlib
from scipy.optimize import minimize

# My modules 
import autoregressiveFunctions as ar
import statFunctions as stat

In [ ]:
importlib.reload(ar) # if autoregressiveFunctions.py is modified 
importlib.reload(stat) # if statFunctions.py is modified 


<module 'statFunctions' from 'c:\\Users\\alber\\python_projects\\fundamentals_of_interest_rates\\statFunctions.py'>

In [6]:
a = np.array([0, 0.5])

y_t = ar.generate_ar(steps=1000, paths=1, a=a, start=0, dist='normal', error_var=1)

100%|██████████| 999/999 [00:00<00:00, 239914.67it/s]

1 different AR(1) processes of 1001 steps have been generated with increments following normal distribution


In [14]:
def fit_ar_ML_normal(y_t: np.array, p: int, dist='normal') -> np.array:

    init_a0 = np.mean(y_t)
    init_coeff = np.array([0.8])
    init_sigma_2 = np.var(y_t)
    x0 = np.concatenate([[init_a0], init_coeff, [init_sigma_2]])

    bounds = [(None,None)] * (p+1) + [(0.1,None)] # No bounds for coeff + Bounds ( >0) for variance

    res = minimize(fun=lik.neg_loglik_normal_ar, x0=x0, args=(y_t,), method='L-BFGS-B', bounds=bounds)
    return res

fit_ar_ML_normal(y_t=y_t, p=1)

  message: CONVERGENCE: NORM OF PROJECTED GRADIENT <= PGTOL
  success: True
   status: 0
      fun: -0.0
        x: [-5.947e-02  8.000e-01  1.391e+00]
      nit: 0
      jac: [ 0.000e+00  0.000e+00  0.000e+00]
     nfev: 4
     njev: 1
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>

In [ ]:
def loglik_normal(y_t: np.ndarray, params) -> float:

    mu = params[0]
    sigma_2 = params[1]

    T = y_t.size
    term_1 = - 0.5 * T * np.log(2 * np.pi)
    term_2 = - 0.5 * T * np.log(sigma_2)
    term_3 = - ((y_t - mu)**2) / (2 * sigma_2)

    return term_1 + term_2 + np.sum(term_3, axis=0) 

In [17]:
init_mu = np.mean(y_t)
init_sigma_2 = np.var(y_t)

x0 = np.array([np.mean(y_t), np.var(y_t)])
bounds = [(None,None), (0.1,None)]

res = minimize(fun=lik.loglik_normal, x0=x0, args=(y_t,), method='L-BFGS-B', bounds=bounds)
print(res)

TypeError: loglik_normal() missing 1 required positional argument: 'sigma_2'